## 对于从HORRIZAN上下载的数据，其只有坐标和速度，如何通过符号回归给出其加速度的函数表达式呢？
#### 根据某篇文献题到的方法，首先我们使用神经网络近似出数据集内的轨迹运动，即加速度的拆分一阶微分方程。通过这个微分方程，我们就可以得到加速度的近似值，从而去寻找函数表达式

### 1. 测试起见，我们使用之前训练好的微分方程。


In [ ]:
import os
import torch
import torch.nn as nn
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# 2. 定义ODE模型
class ODEFunc(nn.Module):
    def __init__(self):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 32),
            nn.Tanh(),
            nn.Linear(32, 32)  
        )
        self.fc1 = nn.Linear(32,20)
        self.fc2 = nn.Linear(20,4)

    def forward(self, t, y):
        y = self.net(y)
        y= torch.sin(y)
        y = self.fc1(y)
        y = torch.tanh(y)
        y = self.fc2(y)

        return y

model = ODEFunc()
model.load_state_dict(torch.load('../improved_data/xy_orbit_model_weights.pth'))

In [ ]:
# 比较一下数据差别

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. 数据读取和预处理
# 加载数据
df = pd.read_csv('../improved_data/xy_orbit.csv')
array = df.to_numpy()
array = array.T
t = array[0]
yy = array[1:]/1e10

print(yy.shape)
# 提取坐标和速度，注意单位一致
x , y, vx, vy = yy[0], yy[1],  yy[2],  yy[3]

# 计算距离 r 和加速度分量
r = np.sqrt(x**2 + y**2)
G = 6.67430e-11 * 86400**2/1e30  # 引力常数, m^3 kg^-1 s^-2（调整单位）
M = 1.989e30  # 太阳质量
GM = G * M

# 加速度分量
x_ddot = -GM * x / r**3
y_ddot = -GM * y / r**3


plt.figure(figsize=(8, 6))
plt.plot(t,x_ddot, color = 'red',label = 'x_ddot')
plt.plot(t,y_ddot, color = 'green',label = 'y_ddot')
plt.xlabel('t')
plt.ylabel('ax or ay (1e10 m/day^2)')
plt.title('acceleration of the System')
plt.legend()
plt.grid()
plt.show()


# 根据模型计算出的加速度
yo = torch.tensor(yy.T, dtype=torch.float32)
tt = model.forward(t,yo)
print(yo.shape)

plt.figure(figsize=(8, 6))
plt.plot(t,yo[:,2], color = 'red',label = 'x_ddot')
plt.plot(t,yo[:,3], color = 'green',label = 'y_ddot')
plt.xlabel('t')
plt.ylabel('ax or ay (1e10 m/day^2)')
plt.title('acceleration of the System')
plt.legend()
plt.grid()
plt.show()

## 观察下来发现，这个微分方程并非学习为二阶，故而需要重新训练一个网络。将方程确定为二阶。

In [2]:
## 1.学习一个微分方程

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchdiffeq import odeint  # 神经ODE库

df = pd.read_csv('../improved_data/xy_orbit.csv')
array = df.to_numpy()
array = array.T
t = array[0]/100
y = array[1:]/1e10

coords = torch.tensor(y.T, dtype=torch.float32)
t_tensor = torch.tensor(t,dtype=torch.float32)

print(coords.shape)

#定义Stan激活函数
class StanActivation(nn.Module):
    def __init__(self, num_neurons):
        """
        初始化 Stan 激活函数。
        :param num_neurons: 每层的神经元数量，用于定义 beta 参数的大小
        """
        super(StanActivation, self).__init__()
        # 使用 nn.Parameter 初始化 beta，使其参与反向传播
        self.beta = nn.Parameter(torch.ones(num_neurons) * 0.1)  # 使用较小值初始化 beta
    
    def forward(self, x):
        """
        Stan 激活函数的前向计算
        :param x: 输入张量
        :return: 应用 Stan 激活函数后的输出张量
        """
        # 确保 beta 参与计算图
        return (1 + self.beta * x) * torch.tanh(x)

# 2. 定义ODE模型
class ODEFunc(nn.Module):
    def __init__(self):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(4,40 ),
            nn.Tanh(),
            nn.Linear(40, 32)  
        )
        self.fc1 = nn.Linear(32,32)
        self.stan = StanActivation(num_neurons=32)
        self.fc2 = nn.Linear(32,2)

    def forward(self,t, y):
        y2 = self.net(y)
        y2= torch.sin(y2)
        y2 = self.fc1(y2)
        y2 = self.stan(y2)
        y2 = self.fc2(y2)
        return torch.cat([y[2].unsqueeze(0), y2[0].unsqueeze(0), y[3].unsqueeze(0), y2[1].unsqueeze(0)])
    

# 早停用函数
def errors(data):
    mean_value = np.mean(data)
    errors = data - mean_value
    absolute_errors = np.abs(errors)
    mae = np.mean(absolute_errors)
    return mae


# 初始条件
G = 6.67430e-11  # 引力常数, m^3 kg^-1 s^-2
M = 1.989e30     # 太阳质量, kg
#r0 = [1.496e11, 0]#初始位置 m
#v0 = [0,30.29e3 ] #初始速度 m/s
#m = 5.972e24     # 地球质量 kg
print(y.shape)

#统一单位
G = G*86400**2 / 1e30
#r0 =r0/1e10
#v0 = v0 * 86400 / 1e10

def energy(Y, G, M):
    x, y, vx, vy = Y[:,0],Y[:,1],Y[:,2],Y[:,3]
    K = (vx**2 + vy**2)/2 - G*M/torch.sqrt(x**2 + y**2)
    return K

def angle(Y,G,M):
    x, y, vx, vy = Y[:,0],Y[:,1],Y[:,2],Y[:,3]
    l = x*vy-y*vx
    return l

energy0 = energy(coords,G,M)
angle0 = angle(coords,G,M)

print(energy0,angle0)
print(energy0.shape,angle0.shape)

class CustomLoss:
    def __init__(self, alpha=0.1, beta=0.1):
        self.alpha = alpha
        self.beta = beta

    def __call__(self, y, y_target):
        trajectory_loss = 30 * torch.mean((y - y_target)**2)
        energy_loss = torch.mean((energy(y,G,M)-energy0)**2)
        angle_loss = torch.mean((angle(y,G,M)-angle0)**2)
        if self.alpha != 0 and self.beta != 0:
            return trajectory_loss  + self.alpha * energy_loss + self.beta * angle_loss
        if self.alpha == 0 and self.beta != 0:
            return trajectory_loss  + self.beta * angle_loss
        if self.beta == 0 and self.alpha != 0:
            return trajectory_loss  + self.alpha * energy_loss
        if self.beta == 0 and self.alpha == 0:
            return trajectory_loss
        
# 超参数设置
loss_value = []
mini_delta = 0.005
sum_ = np.array([0,0,0,0,0])

# 定义训练函数
def train(model,optimizer,criterion,epochs):
    model.train()
    for epoch in range(epochs+1):
        optimizer.zero_grad()
        
        # 神经ODE求解
        pred_y = odeint(model, coords[0], t_tensor)
        
        # 计算损失
        loss = criterion(pred_y, coords)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()
        
        # 打印损失
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

        #设置早停
        loss_value.append(loss.item())
        if epoch > 5:
            for i in range(5):
                sum_[i] = loss_value[epoch]-loss_value[epoch-i-1]
        err = errors(sum_)

        if epoch == epochs:
            torch.save(model.state_dict(), 'ax_ay_model_params.pth')

        if loss.item() < 0.01 and err < 0.001 :
            torch.save(model.state_dict(), 'ax_ay_model_params.pth')
            break

torch.Size([8760, 4])
(4, 8760)
tensor([-0.0331, -0.0331, -0.0331,  ..., -0.0331, -0.0331, -0.0331]) tensor([3.8497, 3.8497, 3.8497,  ..., 3.8497, 3.8497, 3.8497])
torch.Size([8760]) torch.Size([8760])


In [3]:
#设置训练：
model = ODEFunc()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
epochs = 2000
loss = CustomLoss(0,0)

In [4]:
train(model,optimizer,loss,epochs)

Epoch 0, Loss: 3073.763916015625
Epoch 100, Loss: 2312.845947265625
Epoch 200, Loss: 1691.0325927734375
Epoch 300, Loss: 1360.8446044921875
Epoch 400, Loss: 1448.8349609375
Epoch 500, Loss: 1340.521240234375
Epoch 600, Loss: 1266.3526611328125
Epoch 700, Loss: 2383.99072265625
Epoch 800, Loss: 1449.817138671875
Epoch 900, Loss: 1333.3883056640625
Epoch 1000, Loss: 1259.3309326171875
Epoch 1100, Loss: 1233.2225341796875
Epoch 1200, Loss: 1213.8011474609375
Epoch 1300, Loss: 1198.086669921875
Epoch 1400, Loss: 1187.991943359375
Epoch 1500, Loss: 1179.8682861328125
Epoch 1600, Loss: 1174.110595703125
Epoch 1700, Loss: 2875.0029296875
Epoch 1800, Loss: 2578.05078125
Epoch 1900, Loss: 2390.654052734375
Epoch 2000, Loss: 2265.22216796875
